In [ ]:
library(fpp3)

In [ ]:
google_2015 <- gafa_stock %>%
    filter(Symbol == "GOOG") %>%
    mutate(Day = row_number()) %>%
    update_tsibble(index = Day)

In [ ]:
google_2015 %>%
    mutate(diff_close = difference(Close)) %>%
    autoplot(diff_close)

In [ ]:
PBS %>%
    filter(ATC2 == "H02") %>%
    summarise(Cost = sum(Cost)/1e6) %>%
    transmute(
        `Sales ($million)` = Cost,
        `Log sales` = log(Cost),
        `Annual change in log sales` = difference(log(Cost), 12),
        `Doubly differenced log sales` = difference(difference(log(Cost), 12)),
    ) %>%
    pivot_longer(-Month, names_to="Type", values_to="Sales") %>%
    mutate(
        Type = factor(Type, levels = c(
          "Sales ($million)",
          "Log sales",
          "Annual change in log sales",
          "Doubly differenced log sales"))
    ) %>%
    ggplot(aes(Month, Sales)) +
    geom_line() +
    facet_grid(vars(Type), scales = "free_y")

In [ ]:
global_economy %>%
    filter(Code == "EGY") %>%
    autoplot(Exports)

In [ ]:
fit <- global_economy %>%
    filter(Code == "EGY") %>%
    model(ARIMA(Exports))

report(fit)

In [ ]:
fit %>%
    forecast(h = 10) %>%
    autoplot(global_economy)

In [ ]:
global_economy %>%
    filter(Code == "EGY") %>%
    ACF(Exports) %>%
    autoplot()

In [ ]:
global_economy %>%
    filter(Code == "EGY") %>%
    gg_tsdisplay(Exports, plot_type = "partial")

In [ ]:
global_economy %>%
    filter(Code == "EGY") %>%
    model(ARIMA(Exports)) %>%
    report()

In [ ]:
caf_economy <- global_economy %>%
    filter(Code == "CAF")

In [ ]:
caf_economy %>% autoplot(Exports)

In [ ]:
caf_economy %>%
    gg_tsdisplay(difference(Exports), plot_type = "partial")

In [ ]:
fit <- caf_economy %>%
      model(
        arima210 = ARIMA(Exports ~ pdq(2,1,0)),
        arima013 = ARIMA(Exports ~ pdq(0,1,3)),
        stepwise = ARIMA(Exports),
        search = ARIMA(Exports, stepwise=FALSE)
      )

fit %>% pivot_longer(!Country, names_to = "Model name", values_to = "Orders")

In [ ]:
glance(fit) %>% arrange(AICc) %>% select(.model:BIC)

In [ ]:
fit %>% 
    select(search) %>%
    gg_tsresiduals()

In [ ]:
fit %>%
    select(search) %>%
    augment() %>%
    features(.innov, ljung_box, lag = 10, dof = 3)

In [ ]:
leisure <- us_employment %>%
    filter(Title == "Leisure and Hospitality", year(Month) > 2000) %>%
    mutate(Employed = Employed / 1000) %>%
    select(Month, Employed)

autoplot(leisure, Employed)

In [ ]:
leisure %>% gg_tsdisplay(
    difference(Employed, 12) %>% difference(), 
    plot_type = "partial",
    lag = 36
)

In [ ]:
fit <- leisure %>%
    model(
        arima012011 = ARIMA(Employed ~ pdq(0,1,2) + PDQ(0,1,1)),
        arima210011 = ARIMA(Employed ~ pdq(2,1,0) + PDQ(0,1,1)),
        auto = ARIMA(Employed, stepwise = FALSE, approx = FALSE)
    )

fit %>% pivot_longer(
    everything(), names_to = "Model name", values_to = "Order"
)

In [ ]:
glance(fit) %>% arrange(AICc) %>% select(.model:BIC)

In [ ]:
fit %>% select(auto) %>% gg_tsresiduals(lag = 36)

In [ ]:
fit %>%
    select(auto) %>%
    augment() %>%
    features(.innov, ljung_box, lag = 36, dof = 6)

In [ ]:
h02 <- PBS %>%
  filter(ATC2 == "H02") %>%
  summarise(Cost = sum(Cost)/1e6)
h02 %>%
  mutate(log(Cost)) %>%
  pivot_longer(-Month) %>%
  ggplot(aes(x = Month, y = value)) +
  geom_line() +
  facet_grid(name ~ ., scales = "free_y") +
  labs(y="", title="Corticosteroid drug scripts (H02)")

In [ ]:
h02 %>% gg_tsdisplay(
    difference(log(Cost), 12), plot_type = "partial", lag = 36
)